In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import stats
from sklearn import metrics,preprocessing,svm

from sklearn.model_selection import cross_val_score, KFold


%matplotlib inline

In [2]:
PATH_TO_DATA = '../data/raw/data.csv'

In [3]:
SEED = np.random.randint(low=0,high=1000,size=1)[0]
SEED

146

In [4]:
df = pd.read_csv(PATH_TO_DATA)

In [5]:
df.head()

,V1,V2,V3,V4,Grupo
0,5.1,3.5,1.4,0.2,G1
1,4.9,3.0,1.4,0.2,G1
2,4.7,3.2,1.3,0.2,G1
3,4.6,3.1,1.5,0.2,G1
4,5.0,3.6,1.4,0.2,G1


In [6]:
X = df[["V1","V2","V3","V4"]].values
y = df[["Grupo"]].values

In [7]:
X.dtype,y.dtype

(dtype('float64'), dtype('O'))

In [81]:
k_fold = KFold(n_splits=10, random_state=SEED,shuffle=True)

accuracies = []

for train_idx, test_idx in k_fold.split(X):
       
    X_train = X[train_idx]
    X_test = X[test_idx]
    y_train = y[train_idx].ravel()
    y_test = y[test_idx].ravel()
    
    scaler = preprocessing.StandardScaler().fit(X_train)
    
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    clf = svm.SVC(kernel='linear')
    clf.fit(X_train_scaled,y_train)
    
    y_pred = clf.predict(X_test_scaled)
    
    accuracy = metrics.accuracy_score(y_test,y_pred)
    
    accuracies.append(accuracy)
    
accuracies = np.array(accuracies)

print(accuracies.mean())    

0.966666666667


Comentários:

- KFold é um mecanismo para fazer validação cruzada (cross validation) para o treinamento do nosso modelo. Isso é uma forma de evitar overfitting (modelar o ruído) e, ao mesmo tempo, nos deixar usar todos os dados do dataset para treinamento.

- Eu usei shuffle=True pois o dataset veio ordenado e, se não for embaralhado, os splits vão ser quase todos dentro da mesma classe.

- Eu usei random_state=SEED para termos reproducibilidade.

- Os dados foram normalizados dentro do split para não haver vazamento de informação. Esse tipo de normalização foi usada pois o nosso dataset não tem muitos outliers.

- Os parâmetros default do classificador SVM foram usados (kernel=RBF).

- Foi usada Acurácia como métrica de sucesso pois o nosso dataset está bem balanceado com relação às classes (há o mesmo número de amostras em cada classe).